<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-부재종-후보군-Dataframe-생성" data-toc-modified-id="1.-부재종-후보군-Dataframe-생성-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>1. 부재종 후보군 Dataframe 생성</a></span></li></ul></div>

In [1]:
import pandas as pd
from tqdm import tqdm
from haversine import haversine

# 1. 부재종 후보군 Dataframe 생성
1. H.convergens
2. C7
3. H.Parenthesis


주변에 c9이 있는지 없는지부터검사.


In [2]:
df = pd.read_csv('../../20220209_07_21_무당벌레관측.csv')
df_c9 = pd.read_csv('../../1.c9_arround_ladybug_presence/20220209_bipunctata_arround_gps_1000.csv')

In [3]:
#gps 변환
df.Latitude = df.Latitude.astype(float)
df.Longitude = df.Longitude.astype(float)

df_gps = []
for i in tqdm(range(len(df))):
    df_gps.append( (df.Latitude[i], df.Longitude[i]) )
    
df['gps'] = df_gps



#c9 gps 변환
df_c9.Latitude = df_c9.Latitude.astype(float)
df_c9.Longitude = df_c9.Longitude.astype(float)

df_c9_gps = []
for i in tqdm(range(len(df_c9))):
    df_c9_gps.append( (df_c9.Latitude[i], df_c9.Longitude[i]) )
    
df_c9['gps'] = df_c9_gps

100%|███████████████████████████████████████████████████████████████████████████| 1487/1487 [00:00<00:00, 74548.25it/s]


In [4]:
#gps 정확도가 1km이상 오차가 있으면 거르기
df = df[df.public_positional_accuracy<=1000].reset_index(drop=True)

In [5]:
#Latitude, Longitude 0으로 기록된 부분들 삭제
df = df[(df.Latitude!=0.0)&(df.Longitude!=0.0)].reset_index(drop=True)

In [6]:
# Absence 종으로 선정될 수 있는 종들만 고르기

df_ab = df[(df.Species=='Harmonia axyridis')|(df.Species=='Coccinella septempunctata')|(df.Species=='Hippodamia convergens')].reset_index(drop=True)

In [7]:
## 주변에 타겟종 있을시 제외
df_ab['c9_500m'] = 0

In [8]:
for i in tqdm(range(len(df_ab))): # c9만 뽑아놓은 데이터 프레임
    for j in range(len(df_c9.gps)):  # 전체 무당벌레 데이터 프레임        
        if haversine(df_ab.gps[i], df_c9.gps[j], unit='km') <= 0.5 and df_c9.Year[j] == df_ab.Year[i]:
            df_ab['c9_500m'][i] += 1

  0%|                                                                               | 8/117423 [00:00<52:41, 37.14it/s]<ipython-input-8-1a53204b015c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ab['c9_500m'][i] += 1
100%|██████████████████████████████████████████████████████████████████████████| 117423/117423 [49:56<00:00, 39.19it/s]


In [23]:
df_ab = df_ab[df_ab['c9_500m']==0].reset_index(drop=True)

In [24]:
# 주가 없다면 거르기
df_ab = df_ab[df_ab.State.isnull()==False].reset_index(drop=True)

In [35]:
hip = df_ab[df_ab.Species=='Harmonia axyridis'].sort_values(by=['Year','Latitude','Longitude']).reset_index(drop=True)
sep = df_ab[df_ab.Species=='Coccinella septempunctata'].sort_values(by=['Year','Latitude','Longitude']).reset_index(drop=True)
conv = df_ab[df_ab.Species=='Hippodamia convergens'].sort_values(by=['Year','Latitude','Longitude']).reset_index(drop=True)


In [36]:
hip.State = hip.State.astype(str)
sep.State = sep.State.astype(str)
conv.State = conv.State.astype(str)

In [37]:
one_idx=[]
many_idx = []
for i in hip.State.unique():
        if i == 'Texas':
            print(i,"삽입전 길이",len(many_idx))
            print(i,"의 개수",len(hip[hip.State==i].index))
            for j in range(len(hip[hip.State==i].index)):
                if (j%120) == 0:
                    many_idx.append(hip[hip.State==i].index[j])
            print(i,"삽입된 후 길이",len(many_idx))
            
        elif len(hip[(hip.State == i)]) <=20:
            one_idx.append(hip[(hip.State == i)].index)
                        
        elif len(hip[(hip.State == i)])<=60:
            print(i,"삽입전 길이",len(many_idx))
            print(i,"의 개수",len(hip[hip.State==i].index))
            for j in range(len(hip[hip.State==i].index)):
                if (j%3) == 0:
                    many_idx.append(hip[hip.State==i].index[j])
            print(i,"삽입된 후 길이",len(many_idx))
            
        elif len(hip[(hip.State == i)])<=120:
            print(i,"삽입전 길이",len(many_idx))
            print(i,"의 개수",len(hip[hip.State==i].index))
            for j in range(len(hip[hip.State==i].index)):
                if (j%5) == 0:
                    many_idx.append(hip[hip.State==i].index[j])
            print(i,"삽입된 후 길이",len(many_idx))
            
            
        elif len(hip[(hip.State == i)])<=500:
            print(i,"삽입전 길이",len(many_idx))
            print(i,"의 개수",len(hip[hip.State==i].index))
            for j in range(len(hip[hip.State==i].index)):
                if (j%20) == 0:
                    many_idx.append(hip[hip.State==i].index[j])
            print(i,"삽입된 후 길이",len(many_idx))
            
            
        elif len(hip[(hip.State == i)])<=1000:
            print(i,"삽입전 길이",len(many_idx))
            print(i,"의 개수",len(hip[hip.State==i].index))
            for j in range(len(hip[hip.State==i].index)):
                if (j%40) == 0:
                    many_idx.append(hip[hip.State==i].index[j])
            print(i,"삽입된 후 길이",len(many_idx))
            
        elif len(hip[(hip.State == i)])<=2000:
            print(i,"삽입전 길이",len(many_idx))
            print(i,"의 개수",len(hip[hip.State==i].index))
            for j in range(len(hip[hip.State==i].index)):
                if (j%60) == 0:
                    many_idx.append(hip[hip.State==i].index[j])
            print(i,"삽입된 후 길이",len(many_idx))
            
        elif len(hip[(hip.State == i)])<=4000:
            print(i,"삽입전 길이",len(many_idx))
            print(i,"의 개수",len(hip[hip.State==i].index))
            for j in range(len(hip[hip.State==i].index)):
                if (j%80) == 0:
                    many_idx.append(hip[hip.State==i].index[j])
            print(i,"삽입된 후 길이",len(many_idx))
                        
        else:
            print(i,"삽입전 길이",len(many_idx))
            print(i,"의 개수",len(hip[hip.State==i].index))
            for j in range(len(hip[hip.State==i].index)):
                if (j%100) == 0:
                    many_idx.append(hip[hip.State==i].index[j])
            print(i,"삽입된 후 길이",len(many_idx))

ff = [element for array in one_idx for element in array]      


hip_1 = hip.iloc[many_idx]
hip_2 = hip.iloc[ff]

hip_fin = pd.concat([hip_1,hip_2]).reset_index(drop=True)

Florida 삽입전 길이 0
Florida 의 개수 2596
Florida 삽입된 후 길이 33
Texas 삽입전 길이 33
Texas 의 개수 8796
Texas 삽입된 후 길이 107
Louisiana 삽입전 길이 107
Louisiana 의 개수 709
Louisiana 삽입된 후 길이 125
Alabama 삽입전 길이 125
Alabama 의 개수 1011
Alabama 삽입된 후 길이 142
Georgia 삽입전 길이 142
Georgia 의 개수 1446
Georgia 삽입된 후 길이 167
California 삽입전 길이 167
California 의 개수 6309
California 삽입된 후 길이 231
Tennessee 삽입전 길이 231
Tennessee 의 개수 1647
Tennessee 삽입된 후 길이 259
North Carolina 삽입전 길이 259
North Carolina 의 개수 2855
North Carolina 삽입된 후 길이 295
Oklahoma 삽입전 길이 295
Oklahoma 의 개수 496
Oklahoma 삽입된 후 길이 320
Missouri 삽입전 길이 320
Missouri 의 개수 855
Missouri 삽입된 후 길이 342
Maryland 삽입전 길이 342
Maryland 의 개수 2287
Maryland 삽입된 후 길이 371
Delaware 삽입전 길이 371
Delaware 의 개수 674
Delaware 삽입된 후 길이 388
New Jersey 삽입전 길이 388
New Jersey 의 개수 1672
New Jersey 삽입된 후 길이 416
New York 삽입전 길이 416
New York 의 개수 4168
New York 삽입된 후 길이 458
Pennsylvania 삽입전 길이 458
Pennsylvania 의 개수 2719
Pennsylvania 삽입된 후 길이 492
Connecticut 삽입전 길이 492
Connecticut 의 개수 731
Connecticut 삽입된 후 길

In [38]:
one_idx=[]
many_idx = []

In [39]:
for i in sep.State.unique():
        if i == 'California' or i == 'Texas':
            print(i,"삽입전 길이",len(many_idx))
            print(i,"의 개수",len(sep[sep.State==i].index))
            for j in range(len(sep[sep.State==i].index)):
                if (j%100) == 0:
                    many_idx.append(sep[sep.State==i].index[j])
            print(i,"삽입된 후 길이",len(many_idx))
            
            
        elif len(sep[(sep.State == i)]) <=10:
            one_idx.append(sep[(sep.State == i)].index)
            
            
        elif len(sep[(sep.State == i)])<=60:
            print(i,"삽입전 길이",len(many_idx))
            print(i,"의 개수",len(sep[sep.State==i].index))
            for j in range(len(sep[sep.State==i].index)):
                if (j%3) == 0:
                    many_idx.append(sep[sep.State==i].index[j])
            print(i,"삽입된 후 길이",len(many_idx))
            
        elif len(sep[(sep.State == i)])<=120:
            print(i,"삽입전 길이",len(many_idx))
            print(i,"의 개수",len(sep[sep.State==i].index))
            for j in range(len(sep[sep.State==i].index)):
                if (j%5) == 0:
                    many_idx.append(sep[sep.State==i].index[j])
            print(i,"삽입된 후 길이",len(many_idx))
            
        elif len(sep[(sep.State == i)])<=200:
            print(i,"삽입전 길이",len(many_idx))
            print(i,"의 개수",len(sep[sep.State==i].index))
            for j in range(len(sep[sep.State==i].index)):
                if (j%6) == 0:
                    many_idx.append(sep[sep.State==i].index[j])
            print(i,"삽입된 후 길이",len(many_idx))
            
            
        elif len(sep[(sep.State == i)])<=500:
            print(i,"삽입전 길이",len(many_idx))
            print(i,"의 개수",len(sep[sep.State==i].index))
            for j in range(len(sep[sep.State==i].index)):
                if (j%20) == 0:
                    many_idx.append(sep[sep.State==i].index[j])
            print(i,"삽입된 후 길이",len(many_idx))
            
            
        elif len(sep[(sep.State == i)])<=1000:
            print(i,"삽입전 길이",len(many_idx))
            print(i,"의 개수",len(sep[sep.State==i].index))
            for j in range(len(sep[sep.State==i].index)):
                if (j%40) == 0:
                    many_idx.append(sep[sep.State==i].index[j])
            print(i,"삽입된 후 길이",len(many_idx))
            
        elif len(sep[(sep.State == i)])<=2500:
            print(i,"삽입전 길이",len(many_idx))
            print(i,"의 개수",len(sep[sep.State==i].index))
            for j in range(len(sep[sep.State==i].index)):
                if (j%60) == 0:
                    many_idx.append(sep[sep.State==i].index[j])
            print(i,"삽입된 후 길이",len(many_idx))
        else:
            print(i,"삽입전 길이",len(many_idx))
            print(i,"의 개수",len(sep[sep.State==i].index))
            for j in range(len(sep[sep.State==i].index)):
                if (j%80) == 0:
                    many_idx.append(sep[sep.State==i].index[j])
            print(i,"삽입된 후 길이",len(many_idx))

        

Texas 삽입전 길이 0
Texas 의 개수 6341
Texas 삽입된 후 길이 64
Louisiana 삽입전 길이 64
Louisiana 의 개수 68
Louisiana 삽입된 후 길이 78
Arizona 삽입전 길이 78
Arizona 의 개수 168
Arizona 삽입된 후 길이 106
South Carolina 삽입전 길이 106
South Carolina 의 개수 222
South Carolina 삽입된 후 길이 118
California 삽입전 길이 118
California 의 개수 7430
California 삽입된 후 길이 193
Alabama 삽입전 길이 193
Alabama 의 개수 148
Alabama 삽입된 후 길이 218
New Mexico 삽입전 길이 218
New Mexico 의 개수 112
New Mexico 삽입된 후 길이 241
Colorado 삽입전 길이 241
Colorado 의 개수 797
Colorado 삽입된 후 길이 261
Illinois 삽입전 길이 261
Illinois 의 개수 198
Illinois 삽입된 후 길이 294
Maryland 삽입전 길이 294
Maryland 의 개수 635
Maryland 삽입된 후 길이 310
Virginia 삽입전 길이 310
Virginia 의 개수 996
Virginia 삽입된 후 길이 335
New York 삽입전 길이 335
New York 의 개수 392
New York 삽입된 후 길이 355
Massachusetts 삽입전 길이 355
Massachusetts 의 개수 216
Massachusetts 삽입된 후 길이 366
Wisconsin 삽입전 길이 366
Wisconsin 의 개수 134
Wisconsin 삽입된 후 길이 389
Michigan 삽입전 길이 389
Michigan 의 개수 235
Michigan 삽입된 후 길이 401
Washington 삽입전 길이 401
Washington 의 개수 592
Washington 삽입된 후 길이 416
Bri

In [40]:
ff = [element for array in one_idx for element in array]

In [41]:
sep_1 = sep.iloc[many_idx]
sep_2 = sep.iloc[ff]
sep_fin = pd.concat([sep_1,sep_2]).reset_index(drop=True)

In [45]:
one_idx=[]
many_idx = []
for i in conv.State.unique():
        if i == 'Texas':
            print(i,"삽입전 길이",len(many_idx))
            print(i,"의 개수",len(conv[conv.State==i].index))
            for j in range(len(conv[conv.State==i].index)):
                if (j%100) == 0:
                    many_idx.append(conv[conv.State==i].index[j])
            print(i,"삽입된 후 길이",len(many_idx))
            
        elif len(conv[(conv.State == i)]) <=20:
            one_idx.append(conv[(conv.State == i)].index)
                        
        elif len(conv[(conv.State == i)])<=60:
            print(i,"삽입전 길이",len(many_idx))
            print(i,"의 개수",len(conv[conv.State==i].index))
            for j in range(len(conv[conv.State==i].index)):
                if (j%3) == 0:
                    many_idx.append(conv[conv.State==i].index[j])
            print(i,"삽입된 후 길이",len(many_idx))
            
        elif len(conv[(conv.State == i)])<=120:
            print(i,"삽입전 길이",len(many_idx))
            print(i,"의 개수",len(conv[conv.State==i].index))
            for j in range(len(conv[conv.State==i].index)):
                if (j%5) == 0:
                    many_idx.append(conv[conv.State==i].index[j])
            print(i,"삽입된 후 길이",len(many_idx))
            
            
        elif len(conv[(conv.State == i)])<=500:
            print(i,"삽입전 길이",len(many_idx))
            print(i,"의 개수",len(conv[conv.State==i].index))
            for j in range(len(conv[conv.State==i].index)):
                if (j%20) == 0:
                    many_idx.append(conv[conv.State==i].index[j])
            print(i,"삽입된 후 길이",len(many_idx))
            
            
        elif len(conv[(conv.State == i)])<=1000:
            print(i,"삽입전 길이",len(many_idx))
            print(i,"의 개수",len(conv[conv.State==i].index))
            for j in range(len(conv[conv.State==i].index)):
                if (j%40) == 0:
                    many_idx.append(conv[conv.State==i].index[j])
            print(i,"삽입된 후 길이",len(many_idx))
            
        elif len(conv[(conv.State == i)])<=2000:
            print(i,"삽입전 길이",len(many_idx))
            print(i,"의 개수",len(conv[conv.State==i].index))
            for j in range(len(conv[conv.State==i].index)):
                if (j%60) == 0:
                    many_idx.append(conv[conv.State==i].index[j])
            print(i,"삽입된 후 길이",len(many_idx))
            
        elif len(conv[(conv.State == i)])<=4000:
            print(i,"삽입전 길이",len(many_idx))
            print(i,"의 개수",len(conv[conv.State==i].index))
            for j in range(len(conv[conv.State==i].index)):
                if (j%70) == 0:
                    many_idx.append(conv[conv.State==i].index[j])
            print(i,"삽입된 후 길이",len(many_idx))
                        
        else:
            print(i,"삽입전 길이",len(many_idx))
            print(i,"의 개수",len(conv[conv.State==i].index))
            for j in range(len(conv[conv.State==i].index)):
                if (j%80) == 0:
                    many_idx.append(conv[conv.State==i].index[j])
            print(i,"삽입된 후 길이",len(many_idx))

ff = [element for array in one_idx for element in array]      


conv_1 = conv.iloc[many_idx]
conv_2 = conv.iloc[ff]

conv_fin = pd.concat([conv_1,conv_2]).reset_index(drop=True)

Florida 삽입전 길이 0
Florida 의 개수 51
Florida 삽입된 후 길이 17
Texas 삽입전 길이 17
Texas 의 개수 2387
Texas 삽입된 후 길이 41
California 삽입전 길이 41
California 의 개수 8210
California 삽입된 후 길이 144
New Mexico 삽입전 길이 144
New Mexico 의 개수 647
New Mexico 삽입된 후 길이 161
Colorado 삽입전 길이 161
Colorado 의 개수 748
Colorado 삽입된 후 길이 180
Washington 삽입전 길이 180
Washington 의 개수 193
Washington 삽입된 후 길이 190
Arizona 삽입전 길이 190
Arizona 의 개수 693
Arizona 삽입된 후 길이 208
Oklahoma 삽입전 길이 208
Oklahoma 의 개수 172
Oklahoma 삽입된 후 길이 217
Kansas 삽입전 길이 217
Kansas 의 개수 78
Kansas 삽입된 후 길이 233
Nevada 삽입전 길이 233
Nevada 의 개수 116
Nevada 삽입된 후 길이 257
Utah 삽입전 길이 257
Utah 의 개수 255
Utah 삽입된 후 길이 270
New York 삽입전 길이 270
New York 의 개수 46
New York 삽입된 후 길이 286
Nebraska 삽입전 길이 286
Nebraska 의 개수 80
Nebraska 삽입된 후 길이 302
Idaho 삽입전 길이 302
Idaho 의 개수 170
Idaho 삽입된 후 길이 311
South Dakota 삽입전 길이 311
South Dakota 의 개수 69
South Dakota 삽입된 후 길이 325
Oregon 삽입전 길이 325
Oregon 의 개수 195
Oregon 삽입된 후 길이 335
Minnesota 삽입전 길이 335
Minnesota 의 개수 162
Minnesota 삽입된 후 길이 344
South Caro

In [46]:
df_fin = pd.concat([sep_fin,hip_fin,conv_fin]).reset_index(drop=True)

In [47]:
df_fin.Species.value_counts()

Harmonia axyridis            1305
Coccinella septempunctata    1118
Hippodamia convergens         749
Name: Species, dtype: int64

In [48]:
df_fin.to_csv("absence_지역선정_1km.csv")